# Process Growth Phases 

This looks at the SQLite Databases generated by Apsim X (Next Gen) for 109 Wheat varieties, 10 differing sow dates for  57,434 locations/sites across Australia.  
Each database file covers one (1) site.

In [ ]:
#Import the required libraries
import sys
import os
import datetime
import sqlite3
import numpy as np
import pandas as pd
import math

In [ ]:
# define the working directories
apsim_sourcedir = "/OSM/CBR/AG_WHEATTEMP/source"
apsim_outfiledir = "/OSM/CBR/AG_WHEATTEMP/work/output"
metfile_sourcedir = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-test/APSIM_run/met"


In [ ]:
dbfile_df = pd.DataFrame(columns=['filename'])
dbfile_df.filename = sorted(apsim_sourcedir+'/'+f for f in os.listdir(apsim_sourcedir) if f.endswith('.db'))
print(dbfile_df.head())
dbname = dbfile_df.filename[0]
print(dbname)

In [ ]:
#these are required for calculations of thermal time
num3hr = int(24 / 3)
#print("num3hr: ", num3hr)    
t_range_fract = []

# pre calculate t_range_fract for speed reasons
for period in range(num3hr):
    calcValue = 0.92105 \
                + 0.1140 * period \
                - 0.0703 * math.pow(period, 2) \
                + 0.0053 * math.pow(period, 3)
    t_range_fract.append(calcValue)

#print("t_range_fract: ", t_range_fract)

In [ ]:
class XYPairs:
    
    def __init__(self, variety):
        self.variety = variety
        #these include the values for wheat
        if variety == "wheat":
            self.X = [0, 26, 37]
            self.Y = [0, 26, 0]
    
    
    def valueIndexed(self, dX):
        '''
        public double ValueIndexed(double dX)
        '''
        did_interpolate = False
        returnVal = np.interp(dX, self.X, self.Y)
        return returnVal
    

In [ ]:
def temp_3hr(tmax, tmin, period):
    '''
    private double temp_3hr(double tmax, double tmin, int period)
    returns the temperature for a 3 hour period.
    a 3 hourly estimate of air temperature
    '''

    if (period < 1):
        print("3 hour period number is below 1")
        
    elif (period > 8):
        print("3 hour period number is above 8")

    # diurnal temperature range for the day (oC)
    diurnal_range = tmax - tmin

    # deviation from day's minimum for this 3 hr period
    t_deviation = t_range_fract[period-1] * diurnal_range

    return tmin + t_deviation


In [ ]:
def Linint3hrlyTemp(tmax, tmin, xp, fp):
    '''
    Eight interpolations of the air temperature are calculated using 
    a three-hour correction factor.
    For each air three-hour air temperature, a value is calculated.
    The eight three-hour estimates are then averaged to obtain the daily value.
    '''

    #Local Variables
    tot = 0.0            #sum_of of 3 hr interpolations
    
    for period in range(1, num3hr):
        #get mean temperature for 3 hr period (oC)
        tmean_3hour = temp_3hr(tmax, tmin, period)
        #tot = tot + ttFn.valueIndexed(tmean_3hour)
        tot = tot + np.interp(tmean_3hour, xp,fp)
        
        #print("tmean_3hour: ", tmean_3hour, " - tot: ", tot)

    return tot / num3hr;


In [ ]:
def linint_3hrly_Temperature(row, xp, fp):
    '''
    Eight interpolations of the air temperature are calculated using 
    a three-hour correction factor.
    For each air three-hour air temperature, a value is calculated.
    The eight three-hour estimates are then averaged to obtain the daily value.
    '''

    #Local Variables
    tot = 0.0            #sum_of of 3 hr interpolations
    
    for period in range(1, num3hr):
        #get mean temperature for 3 hr period (oC)
        tmean_3hour = temp_3hr(row['maxTemp'], row['minTemp'], period)
        #tot = tot + ttFn.valueIndexed(tmean_3hour)
        tot = tot + np.interp(tmean_3hour, xp,fp)
        
        #print("tmean_3hour: ", tmean_3hour, " - tot: ", tot)

    return tot / num3hr;


In [ ]:

#Need to call Linint3hrlyTemp

#print(ttFn.X)
#print(ttFn.Y)
#metData['ApsimTT'] = metData.apply(lambda x: Linint3hrlyTemp(metData['maxTemp'], metData['minTemp'], ttFn), axis=1)
#outVal = Linint3hrlyTemp(31.8, 13.4, ttFn)
#print(outVal)

#print(metData.head(5))

                              

In [ ]:
def read_ApsimWeather(filename, lat):
    '''
    Reads an apsim weather ('.met') file, removes the header information,
    calculates and adds a date column (based on year and day), and the
    average temperature (based on maxt and mint).
    '''
    import math

    xp = [0, 26, 37]
    fp = [0, 26, 0]
    
    lineNo = 0
    with open(filename, "r") as f:
        for line in f:
            lineNo = lineNo + 1
            if line.startswith('year'):
                break;

    # return the data using the starting line no (determined above)
    # original column names=['year','day', 'radn', 'maxt', 'mint', 'rain']
    metData = pd.read_table(filename, sep='\s+', header=None, skiprows=lineNo+1,
                            names=['year','dayofYear', 'radiation', 'maxTemp', 'minTemp', 'rain'])
    
    # add the calculated columns
    metData['runDate'] = pd.to_datetime(metData['year'].astype(str) + " " + metData['dayofYear'].astype(str), format="%Y %j")

    # this may need to be the thermal time, not just average temp
    metData['avgTemp'] = (metData['maxTemp'] + metData['minTemp']) / 2

    #convert the radiation from MJ/m2/day to Photosynthetically active radiation (PAR)
    metData['PARIO'] = metData['radiation'] * 0.47

    #convert the measurement unit for the radiation from MJ/m2/day to J/m2/day
    metData['radnJ'] = metData['radiation'] * 1000000


    # calculation the day length
    radians = math.pi/180
    lambdaRadians = float(latitude) * radians

    sinLAT = math.sin(lambdaRadians)
    cosLAT = math.cos(lambdaRadians)
    sinDMC = radians * 23.45

    #print("radians: ", radians)
    #print("lambdaRadians: ", lambdaRadians)
    #print("sinLAT: ", sinLAT)
    #print("cosLAT: ", cosLAT)
    #print("sinDMC: ", sinDMC)    
    
    metData['sinDEC'] = -sinDMC * np.cos(2 * math.pi * (metData['dayofYear'] + 10) / 365)
    metData['cosDEC'] = np.sqrt(1 - (metData['sinDEC'] * metData['sinDEC']))
    metData['a'] = sinLAT * metData['sinDEC']
    metData['b'] = cosLAT * metData['cosDEC']

    metData['daylength'] = 12 * (1 + (2 / math.pi) * np.arcsin(metData['a']/metData['b']))

    # calculate the Fraction Disfused Radiation (FDR)
    metData['hour'] = np.mod(metData['dayofYear'], 1) * 24
    metData['sinB'] = metData['a'] + metData['b'] * np.cos(2 * math.pi * (metData['hour'] - 12) / 24)
    metData['SC'] = 1367 * (1 + 0.033 * np.cos(2 * math.pi * (metData['dayofYear'] - 10) / 365))
    metData['sinINT'] = metData['a'] * metData['daylength'] + (24 * metData['b'] / math.pi) * \
                        np.cos((math.pi / 2) * ((metData['daylength'] / 12) - 1))

    metData['Ta'] = metData['radnJ'] / (metData['sinINT'] * 3600 * metData['SC'])
    metData['fracDiffusedRadn'] = metData['Ta'] * -1.6 +1.32

    # calculate the Evapotranspiration
    metData['vpsl'] = 238.102 * 17.32491 * ((metData['minTemp'] + metData['maxTemp']) /2) / \
                      (((metData['minTemp'] + metData['maxTemp']) / 2) + 238.102) ** 2
    metData['ETpt'] = 1.26 * (metData['radnJ']  * (metData['vpsl'] / (metData['vpsl'] + 0.067))) / 2454000

    metData['ApsimTT'] = metData.apply(lambda x: Linint3hrlyTemp(x['maxTemp'], x['minTemp'], xp, fp), axis=1)
    

    # sort the columns to be a little more logical
    #cols=['year', 'dayofYear', 'runDate', 'dayLength', 'maxTemp', 'minTemp', 'avgTemp', 'ApsimTT', 'rain', \
    #      'PARIO', 'fracDiffusedRadn', 'vpsl', 'ETpt']
    #metData = metData[cols]    
    
    return metData


In [ ]:
def get_simulation_details(dbname):
    '''
    Opens the specified SQL Database and extracts the 'Name' details from the Simulation
    Table, and splits it to Simulation ID, Longitude, Latitude, Variety, SowDate, and
    returns ad dataframe.
    '''

    # connect to the Database
    con = sqlite3.connect(dbname)
    cur = con.cursor()

    # get contents of the _Simulation Table
    strSql = "SELECT ID as SimulationID, Name FROM _Simulations"
    dfSim = pd.read_sql_query(strSql, con, index_col = 'SimulationID')

    # split the 'Name' field into long, lat, variety and sowdate columns
    dfSim[['long','lat','variety','sowdate']] = \
    dfSim['Name'].str.extract("^(?P<long>\d+)_(?P<lat>-?\d+)_(?P<variety>\S+)_(?P<sowdate>\d+-\S+)$", expand=True)

    # format the columns
    pd.options.display.float_format = '{:,.2f}'.format
    dfSim['long'] = dfSim['long'].astype(float) / 100
    dfSim['lat'] = dfSim['lat'].astype(float) / 100

    # create a SimId column (as the original SimulationID is now an index column)
    dfSim['SimID'] = dfSim.index 

    return dfSim


In [ ]:
def get_report_details(dbname):
    '''
    Opens the specified SQL Database and extracts the details from the Report
    Table, formats the columns correctly and returns a dataframe
    '''

    # connect to the Database
    con = sqlite3.connect(dbname)
    cur = con.cursor()

    # get contents of the Report Table
    #strSql = "SELECT SimulationID, substr([Clock.Today], 1, 10) as runDate, \
    #      [Wheat.Leaf.LAI] as LAI, [Wheat.AboveGround.Wt] as Biomass, \
    #      [Wheat.Grain.Wt] as Yield, [Wheat.Phenology.Zadok.Stage] as ZadokStage, \
    #      [Wheat.WaterSupplyDemandRatio] as WaterSupplyDemandRatio, \
    #      [Wheat.Root.NUptake] as RootNUptake, [Wheat.Leaf.Fn] as LeafFn \
    #      FROM Report \
    #      ORDER BY SimulationID, runDate"
    
    #Need to exclude consider CUTOFFS

    #do not need all of the columns, will cut this down from the get go
    strSql = "SELECT SimulationID, substr([Clock.Today], 1, 10) as runDate, \
          [Wheat.Leaf.LAI] as LAI, [Wheat.AboveGround.Wt] as Biomass, \
          [Wheat.Grain.Wt] as Yield, [Wheat.Phenology.Zadok.Stage] as ZadokStage, \
          [Wheat.WaterSupplyDemandRatio] as WaterSupplyDemandRatio \
          FROM Report \
          ORDER BY SimulationID, runDate"    
    #print(datetime.datetime.now())
    dfReport = pd.read_sql_query(strSql, con, index_col="SimulationID" )
    #print(datetime.datetime.now())

    # format the date columns
    dfReport['runDate'] = pd.to_datetime(dfReport['runDate'], format="%Y-%m-%d")

    # create the SimId column
    dfReport['SimID'] = dfReport.index

    return dfReport


In [ ]:
def get_filename(dbname):
    '''
    Takes the full path and filename for the database file, and creates the filename
    that is used for the weather file, and to save the output.

    Note:  cannot use the db filename as it doesn't have the long & lat that we require
           need to manipulate the filename to add the underscrore '_' char
           ONLY need to allow for negative (south) latitudes
    '''
    filename = os.path.basename(dbname)
    filename = os.path.splitext(filename)[0]
    nameparts = filename.split('-')
    filename = nameparts[0] + '_-' + nameparts[1]
    lat = '-' + nameparts[1]

    return filename, lat


In [ ]:
def get_weather_details(filename, latitude):
    '''
    Retrieves the weather data for the location (long,lat) specified in the dbname,
    formats the data, and returns a dataframe
    '''

    fullfilename = metfile_sourcedir + "/c_" + filename + ".met"
    dfWeather = read_ApsimWeather(fullfilename, latitude)

    return dfWeather


## Process the data
The following forms a single function:  def process_Apsim_dbfile(dbname):  
#### NOTE:  dbname is defined at the top

In [ ]:
print("processing file: ", dbname)
print("started at ", datetime.datetime.now())
filename, latitude = get_filename(dbname)

print("filename: ", filename)
print("latitude: ", latitude)


In [ ]:
# retrieve the weather data from the weather '.met' file
dfWeather = get_weather_details(filename, latitude)
print(dfWeather.shape)
print(dfWeather.head(5))


In [ ]:
#this is so that we can check the calculations and see if everything is correct
#outfilename = apsim_outfiledir + "/" + filename + "_calcs.csv"
#dfWeather.to_csv(outfilename, encoding='utf-8', index=False)


In [ ]:
#this will be part of the get_weather_details functionality, but is commented out so that I can check things
cols=['year', 'dayofYear', 'runDate', 'daylength', 'maxTemp', 'minTemp', 'avgTemp', 'ApsimTT', 'rain', \
      'PARIO', 'fracDiffusedRadn', 'vpsl', 'ETpt']
dfWeather = dfWeather[cols] 
dfWeather

In [ ]:
# retrieve the Simulation Details from the DB._Sumulation table
dfSim = get_simulation_details(dbname) 
print(dfSim.shape)
print(dfSim.head(5))


In [ ]:
# retrieve the Details from the DB.Report table
dfReport = get_report_details(dbname)
print(dfReport.shape)
print(dfReport.head(5))


In [ ]:
# combine the report data with the weather data
dfCombined = dfReport.merge(dfWeather, on='runDate', how='left')
dfCombined
# filter the data based on the information we want
#filterCols = ['SimID', 'runDate', 'ZadokStage', 'avgTemp']
#dfSubData = dfCombined[filterCols]


In [ ]:
# combine the data with the Simulation details, so that we can get the sow date
# and filter it again
dfCombined = dfCombined.merge(dfSim, on="SimID", how='left')
#filterCols = ['SimID', 'runDate', 'ZadokStage', 'avgTemp', 'sowdate']
#dfSubData = dfSubData[filterCols]
dfCombined

In [ ]:
filterCols=['SimID', 'year', 'dayofYear', 'runDate', 'daylength', 'maxTemp', 'minTemp', 'avgTemp', \
            'ApsimTT', 'rain', 'PARIO', 'fracDiffusedRadn', 'vpsl', 'ETpt', 'LAI', 'Biomass', 'Yield', \
            'ZadokStage', 'WaterSupplyDemandRatio', 'long', 'lat', 'variety', 'sowdate' ]

dfSubData = dfCombined[filterCols]
dfSubData


In [ ]:
#??? 
#do we need to re-order the data here so that it is long, lat, variety, sowdate, runDate


In [ ]:
# create a sowing date (with current year)
dfSubData['sowingdate'] = dfSubData['sowdate'] + '-' + dfSubData['runDate'].dt.year.map(str)
dfSubData['sowingdate'] = pd.to_datetime(dfSubData['sowingdate'], format="%d-%b-%Y")

In [ ]:
# now calculate the cumulative temp info for each simulation 
# 90 is maturity (start of ripening stage)
#& (dfSubData['ZadokStage'] <= 90)

dfSubData['tempavgTemp'] = dfSubData['avgTemp'].where((dfSubData['runDate'] >= dfSubData['sowingdate']) 
                                                      & (dfSubData['ZadokStage'] > 0)
                                                      & (dfSubData['ZadokStage'] <= 90), 0)
dfSubData['TTAfterSowing'] = dfSubData.groupby(by=['SimID','sowingdate'])['tempavgTemp'].cumsum()
dfSubData

In [ ]:
# filter the data on the tempavgTemp column
newData = dfSubData[dfSubData['tempavgTemp'] > 0]
#newData1 = newData.groupby(['SimID','sowdate'])['TTAfterSowing'].max().reset_index()
newData1 = newData.groupby(['SimID','sowingdate'])['TTAfterSowing'].max().reset_index()


In [ ]:
#newData2 = newData1.groupby(['SimID','sowdate'])['TTAfterSowing'].mean().reset_index()
newData2 = newData1.groupby(['SimID','sowingdate'])['TTAfterSowing'].mean().reset_index()


In [ ]:
#create bins for the phases
bins = [0, 7, 10, 31, 39, 71, 87, 90, 120]
group_names = ['01_Germinating', '02_Emerging', '03_Vegetative', '04_StemElongation',
               '05_GrainSet', '06_GrainFilling', '07_Maturing', '08_Ripening']
dfSubData['phases'] = pd.cut(dfSubData['ZadokStage'], bins, labels=group_names)
dfSubData

#might need to 

### Need to determine the phases: 
•	length (no of days in phase), minTemp, maxTemp, and avgTemp,  
•	the cumulativeAvgTemp at the end of each phase, (needs to be called TTAfterSowing)  
•	the counts for number of day where temperatures are below or above specified values (refer to SIP_Temp discussion.docs for further details)  
•	the average and cumulative rainfall   
•	average radiation, cumulative radiation  
•	average watersupplydemandratio


In [ ]:
# eliminate those records where the phase is Nan
#dfSubData.dropna(subset=['phases'], inplace=True)
#dfSubData

In [ ]:
#Create a function to get the stats for group
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'mean': group.mean(), 'count': group.count()}

In [ ]:
#should I add a column that can be a concatinated key
dfSubData['SimIdSowingdatePhase'] = dfSubData['SimId'].map(str) + \ 
    dfSubData['sowingdate'].map(str) + dfSubData['phases']

#apply the get_stats function to each phase bin
dfSubData['ZadokStage'].groupby(dfSubData['SimIdSowingdatePhase']).apply(get_stats).unstack()
dfSubData,

In [ ]:
#how many days in each of the phases
newData1.groupby(['SimIdSowingdatePhase'].count().reset_index()

#what is the minimimum temperature for the phase
newData1.groupby(['SimIdSowingdatePhase'])['minTemp'].min().reset_index()
#what is the maximum temperature for the phase
newData1.groupby(['SimIdSowingdatePhase'])['maxTemp'].max().reset_index()

 newData1.groupby(['SimIdSowingdatePhase'])['biomass'].max().reset_index()

#what is the average of average temp for the phase
newData1.groupby(['SimIdSowingdatePhase'])['avgTemp'].mean().reset_index()

#what is the average of average daily radiation for the phase
#newData1.groupby(['SimIdSowingdatePhase'])['radn'].mean().reset_index()

#what is the cumulative rainfall for each phase
newData1.groupby(by=['SimIdSowingdatePhase'])['rain'].cumsum()

newData1.groupby(by=['SimIdSowingdatePhase'])['ETpt'].cumsum()

newData1.groupby(by=['SimIdSowingdatePhase'])['PARIO'].mean().reset_index()
newData1.groupby(by=['SimIdSowingdatePhase'])['PARIO'].cumsum()

newData1.groupby(by=['SimIdSowingdatePhase'])['PQDay'].mean.reset_index()
newData1.groupby(by=['SimIdSowingdatePhase'])['daylengh'].mean.reset_index()

#need to get the values for the last day in each phase:
startdate, enddate, DAS, LAI, Biomass, Yield
                
                 
#what is the average of average watersupplydemandratio for the phase
newData1.groupby(['SimIdSowingdatePhase'])['watersupplydemandratio'].mean().reset_index()


In [ ]:
#get the counts of various miniumum temperatures
newData1[newData1['minTemp'] <= 0].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['minTemp'] <= -1].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['minTemp'] <= -2].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['minTemp'] <= -3].groupby['SimIdSowingdatePhase'].count()


In [ ]:
#get the counts of various maximum temperatures
newData1[newData1['maxTemp'] >= 30].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['maxTemp'] >= 32].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['maxTemp'] >= 34].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['maxTemp'] >= 36].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['maxTemp'] >= 38].groupby['SimIdSowingdatePhase'].count()
newData1[newData1['maxTemp'] >= 40].groupby['SimIdSowingdatePhase'].count()


In [ ]:
# need to add back in the longitude, latitude, variety from dfSim
newData2 = newData2.merge(dfSim, on=['SimID', 'sowdate'], how='left')
filterCols = ['SimID', 'long', 'lat', 'variety', 'sowdate', 'TTAfterSowing']
newData2 = newData2[filterCols]


In [ ]:
outfilename = apsim_outfiledir + "/" + filename + "_zadok.csv"
newData2.to_csv(outfilename, encoding='utf-8', index=False)
